In [1]:
import geopandas as gpd
import pandas as pd
import os
import io
import requests
import boto3

This script intitate the dataframes that will store the different indicators calauclated for urbanshift cities.
- Input: geojson file containing all the cities/sub-cities concerned 
- Output: csv file containing the list of cities and whre the indicators will be stored

Run the script only when you want to initiate all the indicators

In [21]:
# connect to s3
aws_credentials = pd.read_csv('C:\\Users\\Saif.Shabou\\OneDrive - World Resources Institute\\Documents\\aws\\credentials.csv')
aws_key = aws_credentials.iloc[0]['Access key ID']
aws_secret = aws_credentials.iloc[0]['Secret access key']

s3 = boto3.resource(
    service_name='s3',
    aws_access_key_id=aws_key,
    aws_secret_access_key=aws_secret
)

# Load all features

In [22]:
# get all urbanshift cities boundaries at different levels
boundaries_cities_urbanshift = gpd.read_file('https://cities-urbanshift.s3.eu-west-3.amazonaws.com/data/boundaries/v_0/boundary-cities-urbanshift.geojson')
boundaries_cities_urbanshift.head()

,geo_id,geo_level,geo_name,geo_parent_name,creation_date,geometry
0,ARG-Buenos_Aires_ADM-1_1,ADM-1,ARG-Buenos_Aires,ARG-Buenos_Aires,2022-06-23,"MULTIPOLYGON (((-58.36618 -34.59744, -58.36609..."
1,ARG-Buenos_Aires_ADM-2_1,ADM-2,José C. Paz,ARG-Buenos_Aires,2022-06-23,"MULTIPOLYGON (((-58.78474 -34.47614, -58.79169..."
2,ARG-Buenos_Aires_ADM-2_2,ADM-2,La Matanza,ARG-Buenos_Aires,2022-06-23,"MULTIPOLYGON (((-58.52920 -34.65475, -58.53370..."
3,ARG-Buenos_Aires_ADM-2_3,ADM-2,Berazategui,ARG-Buenos_Aires,2022-06-23,"MULTIPOLYGON (((-58.01673 -34.78019, -58.01683..."
4,ARG-Buenos_Aires_ADM-2_4,ADM-2,Vicente López,ARG-Buenos_Aires,2022-06-23,"MULTIPOLYGON (((-58.48032 -34.48861, -58.48036..."


# Create empty indicator dataframe

In [23]:
# create the table
cities_indicators_df = pd.DataFrame(boundaries_cities_urbanshift.drop(columns=['geometry','creation_date']))
cities_indicators_df.head()

,geo_id,geo_level,geo_name,geo_parent_name
0,ARG-Buenos_Aires_ADM-1_1,ADM-1,ARG-Buenos_Aires,ARG-Buenos_Aires
1,ARG-Buenos_Aires_ADM-2_1,ADM-2,José C. Paz,ARG-Buenos_Aires
2,ARG-Buenos_Aires_ADM-2_2,ADM-2,La Matanza,ARG-Buenos_Aires
3,ARG-Buenos_Aires_ADM-2_3,ADM-2,Berazategui,ARG-Buenos_Aires
4,ARG-Buenos_Aires_ADM-2_4,ADM-2,Vicente López,ARG-Buenos_Aires


# Upload in aws s3

In [27]:
# upload to aws
key_data = 'indicators/cities_indicators.csv'
bucket_name = 'cities-urbanshift' 
cities_indicators_df.to_csv(
    f"s3://{bucket_name}/{key_data}",
    index=False,
    storage_options={
        "key": aws_key,
        "secret": aws_secret
    },
)

In [28]:
# make it public
object_acl = s3.ObjectAcl(bucket_name,key_data)
response = object_acl.put(ACL='public-read')